### Project 01 : Acquiring and processing information on world's largest banks
- Task 1: Logging function
- Task 2 : Extraction of data
- Task 3 : Transformation of data
- Task 4: Loading to CSV
- Task 5: Loading to Database
- Task 6: Function to Run queries on Database
- Task 7: Verify log entries

In [1]:
#installing some main libraries
from io import StringIO
import requests
from bs4 import BeautifulSoup
import sqlite3
from datetime import datetime
import pandas as pd

#### Step 0 Maintaining a Log File:

In [9]:
def log_progress(message):
    with open('logs/code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}\n')

#### Step 1: Extraction of Data:

In [3]:
def extract(url, table_attribs):
    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    table = soup.find('span', string = table_attribs).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]

    log_progress('Data extraction complete. Initializing Transformation process')

    return df

#### Step 2: Transformation of Data:

In [4]:
def transform(df, csv_path):
    exchange_rate = pd.read_csv('./input/exchange_rate.csv', index_col = 0).to_dict()['Rate']

    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['GBP'], 2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['EUR'], 2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['INR'], 2)

    log_progress('Data transformation complete. Initiating Loading process')

    return df
    

#### Step 2: Loading:

Loading data to csv

In [5]:
def load_to_csv(df, output_path):
    df.to_csv(output_path)

    log_progress('Data save to csv file')

Loading data to SQL database

In [6]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists = 'replace', index=False)

    log_progress('Data loaded into Database as a table, Executing queries...')

def run_query(query_statement, sql_connection):
    cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    result = cursor.fetchall()

    log_progress('Process Complete')

    return result

In [10]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'

    log_progress('Preliminaries complete.Initiating Etl process')
    df = extract(url, 'By market capitalization')
    transform(df, './data/exchange_rate.csv')
    load_to_csv(df, output_csv_path)

    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)

        run_query('SELECT * FROM Largest_banks', conn)
        run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', conn)
        run_query('SELECT "Bank name" FROM Largest_banks LIMIT 5', conn)


print(df)

   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2                          Bank of America                    231.52   
2     3  Industrial and Commercial Bank of China                    194.56   
3     4               Agricultural Bank of China                    160.68   
4     5                                HDFC Bank                    157.91   
5     6                              Wells Fargo                    155.87   
6     7                        HSBC Holdings PLC                    148.90   
7     8                           Morgan Stanley                    140.83   
8     9                  China Construction Bank                    139.82   
9    10                            Bank of China                    136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215.31    

In [8]:
!pip install lxml